# CLASSIFICAÇÃO DE FAKE NEWS USANDO DEEP LEARNING



Nesta prática iremos classificar um texto a partir de algoritmos de classificação baseado em Redes Neurais Recorrentes (RNN), em especifico Long Short Term Memory (LSTM). Para resolução do problema de classificação, passaremos por algumas etapas, conforme discutido em nossos estudos.

## Por que usar o LSTM para classificação de texto?
Podemos classificar textos a partir do Processamento de Linguagem Natural e diferentes Algoritmos de Classificação baseados em Deep Learning como LSTMs e CNNs.

Existem muitos algoritmos de classificação clássicos como Árvores de Decisão, Random Forest, SVM, que podem fazer um bom trabalho, então por que usar LSTM para classificação?

> Uma boa razão para usar o LSTM é que ele é eficaz na memorização de informações importantes. Se olharmos e outras técnicas de classificação de redes não neurais, elas são treinadas em várias palavras como entradas separadas que são apenas palavras sem significado real como uma frase, e ao prever a classe dará a saída de acordo com as estatísticas e não de acordo com o significado. Isso significa que cada palavra é classificada em uma das categorias.

> Isso ocorre de maneira diferente em LSTM. No LSTM, podemos usar uma string de várias palavras para descobrir a classe à qual ela pertence. Isso é muito útil ao trabalhar com Processamento de Linguagem Natural. Se usarmos camadas apropriadas de incorporação e codificação em LSTM, o modelo será capaz de descobrir o significado real na string de entrada e fornecer a classe de saída mais precisa. O código a seguir elaborará a ideia de como a classificação de texto é feita usando LSTM.

Para a construção do modelo e testes, você pode baixar os dados em https://www.kaggle.com/c/fake-news/data#

## Referencias
> https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

> 

# Importar bibliotecas


In [ ]:
import pandas as pd

## Importar e verificar os dados

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/PLN/train.csv')

'''
# Upload from google drive
from google.colab import files
uploaded = files.upload()
print("len(uploaded.keys():", len(uploaded.keys())) 

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn]))) 
'''

In [ ]:
df.head()

In [ ]:
### Drop Nan Values
df=df.dropna()
## Get the Independent Features
X=df.drop('label',axis=1)
## Get the Dependent features
y=df['label']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000

## One-hot representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)

## Dataset Pre processing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
# print corpus content
print(corpus)

In [ ]:
# one hot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

## Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[0]

## Creating model

In [ ]:

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

## Adicionando o Dropout

In [ ]:
# Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model1.summary())

In [ ]:
len(embedded_docs), y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y) 

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.15, random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
# Finally Training
train_model=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
print(y_test)

## Predição e avaliação do modelo


In [ ]:
#y_pred1=model.predict_classes(X_test)
y_pred1=np.argmax(model.predict(X_test), axis=-1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_model.history['accuracy'],'b',label='train_accuracy')
plt.plot(train_model.history['val_accuracy'],'r',label='val_accuracy')
plt.legend()

In [ ]:
# pred_val=np.argmax(model.predict(x_val), axis=-1)
y_pred1=np.argmax(model.predict(X_test), axis=-1)

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

cm= confusion_matrix(y_test,y_pred1)
plot_confusion_matrix(cm, figsize=(5,5))